In [ ]:
import pandas as pd

# Define the path to your CSV file
#csv_file_path = 'data\loss_wheatwinter_countylevel_2000_2022.csv'
csv_file_path = r'data/usda_loss_wheatwinter_countylevel_2015_2022.csv'

# Load the CSV file into a dataframe
df = pd.read_csv(csv_file_path)
df.head(20)

In [ ]:
import ee
import geemap
import pandas as pd
import folium

try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

state = '05'
year = str(2015)
county = str('Conway')

# Define the boundary of Thomas County, Kansas
countyBoundary = ee.FeatureCollection('TIGER/2018/Counties') \
    .filter(ee.Filter.eq('STATEFP', state)) \
    .filter(ee.Filter.eq('NAME', county)) 


start = year+'-01-01'
end = year+'-12-31'
# Load the CropScape data for the year 2022
cropscape = ee.ImageCollection('USDA/NASS/CDL') \
    .filter(ee.Filter.date(start, end)) \
    .first()

# Select winter wheat (crop code: 3) from the CropScape data
winterWheat = cropscape.select('cropland') \
    .eq(24) \
    .selfMask()

# Clip the winter wheat layer to Thomas County boundary
winterWheatClip = winterWheat.clip(countyBoundary)


# Convert the clipped layer to a FeatureCollection
winterWheatFC = winterWheatClip.reduceToVectors(
    geometry=countyBoundary,
    scale=30,
    geometryType='polygon',
    eightConnected=False,
    labelProperty='class'
)

In [ ]:

# Create a map and add the clipped NDVI map
Map = geemap.Map(basemap = 'SATELLITE')
Map.centerObject(countyBoundary, 10)
palette = ['blue', 'white', 'green']
# Visualize the Landsat composite
visParams = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}  # Adjust the visualization parameters as needed

#Map.addLayer(clippedNDVI.select('nd'), {'min': -1, 'max': 1, 'palette': palette}, 'Clipped NDVI')
Map.addLayer(countyBoundary, {'color': 'FF000000'}, 'County Boundary')
#Map.addLayer(winterWheatFC, {'color': 'FF000000'}, 'Winter Wheat FC')
#Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Combined Image')

# Display the map
Map.addLayerControl()
Map

In [1]:
import pandas as pd

# Define the path to your CSV file
csv_file_path = r'data/usda_loss_wheatwinter_countylevel_2015_2022.csv'

# Load the CSV file into a dataframe
df = pd.read_csv(csv_file_path)

In [2]:
import multiprocessing
from get_ndvi_serie import get_ndvi_serie
import ee

ee.Reset

for j in range(4100, 5000, 100):
    indices = range(j, j+100)

    if __name__ == '__main__':
        with multiprocessing.Pool(processes=25) as pool:
            result = pool.map(get_ndvi_serie, [df.iloc[i] for i in indices])
    
    cols = result[0].index
    new_pd = pd.DataFrame(index=cols).T
    for i in range(len(result)):
        new_pd.loc[i] = result[i].values
    new_pd.to_csv(r'data/data_part/'+str(j)+'_'+str(j+99)+'_2015_2022.csv', index=False)

In [ ]:
import os
import glob
import pandas as pd

# Set the directory containing the CSV files
csv_directory = 'data/data_part'

# Get a list of all CSV files in the directory
csv_files = glob.glob(os.path.join(csv_directory, '*.csv'))

# Initialize an empty DataFrame to store the combined data
combined_data = pd.DataFrame()

# Loop through each CSV file and append its contents to the combined_data DataFrame
for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    combined_data = combined_data.append(df, ignore_index=True)

# Save the combined data to a new CSV file
combined_data.to_csv('data/combined_data_4000_5000.csv', index=False)

print('Combination complete. The combined data has been saved.')


In [ ]:
import ee
import geemap

try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

countyBoundary = ee.FeatureCollection('TIGER/2018/Counties') \
    .filter(ee.Filter.eq('STATEFP', '55')) \
    .filter(ee.Filter.eq('NAME', 'Dane'))

# Create a map and add the clipped NDVI map
Map = geemap.Map(basemap = 'SATELLITE')
Map.centerObject(countyBoundary, 10)

Map.addLayer(countyBoundary, {'color': 'FF000000'}, 'County Boundary')

# Display the map
Map.addLayerControl()
Map

In [ ]:
from get_ndvi_serie import get_ndvi_serie
import ee

try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

res = get_ndvi_serie(df.iloc[7114])

cols = res.index
new_pd = pd.DataFrame(index=cols).T
new_pd.loc[0] = res.values
new_pd